In [3]:
import os
if os.name == 'nt':
    !dir "../dataset"
    !dir "../model"
else:
    !dir "../dataset"
    !dir "../model"

 Volume in drive C has no label.
 Volume Serial Number is CACB-8726

 Directory of C:\Users\L\Desktop\work\dataset

02/01/2019  08:55 AM    <DIR>          .
02/01/2019  08:55 AM    <DIR>          ..
02/01/2019  12:34 AM         4,124,645 neg.txt
02/01/2019  08:52 AM         7,948,509 new_bilstm_neg_tok.pkl
02/01/2019  08:52 AM        19,913,070 new_bilstm_pos_tok.pkl
02/01/2019  01:45 AM         7,722,752 new_neg_tok.pkl
02/01/2019  01:45 AM        19,505,617 new_pos_tok.pkl
02/01/2019  12:34 AM        10,450,593 pos.txt
02/01/2019  12:57 AM        15,330,064 vc_redist.x64.exe
11/08/2018  12:30 AM    <DIR>          wongnai
11/08/2018  12:30 AM           629,380 wongnai_1.csv
11/08/2018  12:30 AM         3,131,209 wongnai_2.csv
11/08/2018  12:30 AM        17,993,664 wongnai_3.csv
11/08/2018  12:30 AM        10,423,335 wongnai_5.csv
              11 File(s)    117,172,838 bytes
               3 Dir(s)  25,067,458,560 bytes free
 Volume in drive C has no label.
 Volume Serial Number is CA

In [4]:
%matplotlib inline

#matplotlib
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import itertools


def plot_accnloss(history):
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)    
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    tick_marks = [0, 1]
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    plt.grid(False)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

#     plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
    plt.show()
    
get_val = lambda arr: list(map(lambda x: x[1], arr))
print('')

In [6]:
import numpy as np
import pandas as pd
import pathlib

from attention import AttentionWithContext

import time, sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import confusion_matrix

from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, Activation, Conv1D, MaxPooling1D, Dropout, Flatten, Bidirectional, GRU, Embedding, LSTM
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import optimizers, utils
from keras import backend as K

import dill as pickle


data_dim = 300
num_classes = 2

input_len = 600
[x_train, x_test, y_train, y_test] = np.load('../dataset/train_test_data.npy')
print(x_train.shape, y_train.shape)
print(x_train.dtype, y_train.dtype)
print(x_test.shape, y_test.shape)
print(x_test.dtype, y_test.dtype)

print(f'pos size = { len(y_test[np.where( y_test[:,0] == 0 )]) }')
print(f'neg size = { len(y_test[np.where( y_test[:,0] == 1 )]) }')

word_vec = np.load('../model/vec.npy')

print(K.floatx()) # should equal float32

Using TensorFlow backend.


(9410, 600) (9410, 2)
float32 int32
(1661, 600) (1661, 2)
float32 int32
pos size = 1112
neg size = 549
float32


In [14]:
dropout = 0.2
dense_dropout = 0.2
rnn_dropout = 0.1
learning_rate = 1e-3
batch_size = 50
epochs = 5

kernel_size = 3
pool_size = 2

model = Sequential()
model.add(Embedding(len(word_vec), 300, input_length=input_len, weights=[word_vec], trainable=True))

# cnn
# model.add(Conv1D(250, kernel_size, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=pool_size))
# model.add(Conv1D(200, kernel_size, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=pool_size))

# 2blstm
model.add(Bidirectional(LSTM(150, return_sequences=True, dropout=dropout, recurrent_dropout=rnn_dropout)))
model.add(Bidirectional(LSTM(128, return_sequences=True, dropout=dropout, recurrent_dropout=rnn_dropout)))
# model.add(Bidirectional(LSTM(128, dropout=dropout, recurrent_dropout=rnn_dropout)))

# 2bgru
# model.add(Bidirectional(GRU(150, return_sequences=True, dropout=dropout, recurrent_dropout=rnn_dropout)))
# model.add(Bidirectional(GRU(128, return_sequences=True, dropout=dropout, recurrent_dropout=rnn_dropout)))
# model.add(Bidirectional(GRU(128, dropout=dropout, recurrent_dropout=rnn_dropout)))

# attn (require return_sequences=True)
model.add(AttentionWithContext())

model.add(Dense(128, activation='tanh'))
model.add(Dropout(dense_dropout))
model.add(Dense(2, activation='softmax'))

adam = optimizers.Adam(lr=learning_rate)
# model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 600, 300)          19500000  
_________________________________________________________________
bidirectional_9 (Bidirection (None, 600, 300)          541200    
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 600, 256)          439296    
_________________________________________________________________
attention_with_context_5 (At (None, 256)               66048     
_________________________________________________________________
dense_9 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 2)                 258       
Total para

In [ ]:
drive_path = '../model/result/'
date = time.strftime('%Y%m%d_%H%M%S')
pathlib.Path(f'{drive_path}{date}').mkdir(parents=True, exist_ok=True)
mon = 'val_acc'
mode_mon = 'max'
file_path = f'{drive_path}{date}/model_{date}.hdf5'
ckpt = ModelCheckpoint(file_path, monitor=mon, verbose=1, save_best_only=True, mode=mode_mon)
early = EarlyStopping(monitor=mon, mode=mode_mon, patience=5)

histories = []

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test), callbacks=[ckpt, early])
del history.model
histories.append(history)

pickle.dump(histories, open(f'{drive_path}{date}/model_{date}_fitting_history.pkl', 'wb'))
        
model_name = f'{drive_path}{date}/model_{date}.h5'
model.save_weights(model_name)
model_json = model.to_json()

with open(f'{drive_path}{date}/model_{date}.json', "w") as json_file:
    json_file.write(model_json)
print("Saved model to disk")


model = load_model(file_path, custom_objects={'AttentionWithContext': AttentionWithContext})
print("Load best model from disk")

rand = np.random.choice(len(x_test), 10, replace=False)
for i in rand:
	pre_x, pre_y = x_test[i], y_test[i]
	pre_x = pre_x.reshape(1, input_len)
	res_y = model.predict_classes(pre_x, batch_size=1)
	print('Prediction: {}'.format(res_y[0]))
	print('Result: {}'.format(str(np.where(pre_y==1.)[0])[1]))

score = model.evaluate(x_test, y_test, batch_size=batch_size)
print("{}: {}".format(model.metrics_names[0], score[0]))
print("{}: {}%".format(model.metrics_names[1], score[1]*100))

y_true = get_val(y_test)
y_predict = model.predict_classes(x_test) 

print(metrics.classification_report(y_true, y_predict, target_names = ['negative', 'positive'], digits=6))

with open(f'{drive_path}{date}/model_{date}_result.txt', 'w') as f:
    model.summary(print_fn=lambda x: f.write(x + '\n'))
    f.write('{}: {}\n'.format(model.metrics_names[0], score[0]))
    f.write('{}: {}%\n'.format(model.metrics_names[1], score[1]*100))
    f.write('{}: {}%\n'.format('Report', metrics.classification_report(y_true, y_predict, target_names = ['negative', 'positive'], digits=6)))
    
    
plot_accnloss(history)

plot_confusion_matrix(confusion_matrix(y_true, y_predict, labels=[1,0]), 
                      classes=['positive', 'negative'], 
                      normalize=True,
                      title='Confusion matrix')

plot_confusion_matrix(confusion_matrix(y_true, y_predict, labels=[1,0]), 
                      classes=['positive', 'negative'], 
                      normalize=False,
                      title='Confusion matrix')


Train on 9410 samples, validate on 1661 samples
Epoch 1/5
9410/9410 [==============================] - 631s 67ms/step - loss: 0.5579 - acc: 0.7320 - val_loss: 0.4003 - val_acc: 0.8224

Epoch 00001: val_acc improved from -inf to 0.82240, saving model to ../model/result/20190207_173135/model_20190207_173135.hdf5
Epoch 2/5
9410/9410 [==============================] - 631s 67ms/step - loss: 0.3489 - acc: 0.8494 - val_loss: 0.2183 - val_acc: 0.9127

Epoch 00002: val_acc improved from 0.82240 to 0.91270, saving model to ../model/result/20190207_173135/model_20190207_173135.hdf5
Epoch 3/5
6300/9410 [===================>..........] - ETA: 3:15 - loss: 0.2093 - acc: 0.9194